# Reading in the Training and Evaluation Data

We will use the msha.csv data set to build and evaluate our autocoder. We begin by dividing it into 3 pieces, each of which will be used for a separate purpose. These pieces are training, validation, and test.

In [ ]:
from __future__ import print_function
from pprint import pprint
import numpy as np
import pandas as pd

# read the csv file into a data frame, a special Python object for representing tabular data
df = pd.read_csv(r'msha.csv', parse_dates=['ACCIDENT_DT'])

# add a column to our data frame with the year the incident occurred
df['YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)

# split the data frame into separate test, validation, and training dataframes based on the year of the incident
df_training = df[df['YEAR'] <= 2011]
df_validation = df[df['YEAR'] == 2012]
df_test = df[df['YEAR'] == 2013]

print('''Data set sizes:
    test: %s
    validation: %s
    training: %s''' % (len(df_test), len(df_validation), len(df_training)))

# Converting inputs to vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of the CountVectorizer object
vectorizer = CountVectorizer()

# Use the narratives in our training data to create the vocabulary that will
# be represented by our feature vectors. This is remembered by the vectorizer.
vectorizer.fit(df_training['NARRATIVE'])

print('Our vectorizer has defined an input vector with %s elements' % len(vectorizer.vocabulary_))
pprint(vectorizer.vocabulary_)

Now that we've defined this mapping of inputs to vector positions we can convert our training data to vector representation.

In [ ]:
# Convert the training narratives into their matrix representation.
x_training = vectorizer.transform(df_training['NARRATIVE'])

print(x_training.shape)

We can also examine the matrix directly but it's not very interesting. Most of the elements of our feature matrix are 0 for any given example.

In [ ]:
n_features = x_training.shape[1]
dense_vector = x_training[0].todense()
print('The vector representing our first training narrative looks like this:', dense_vector)
print('Only %s of the %s elements in this vector are nonzero' % (np.count_nonzero(dense_vector), n_features))

In [ ]:
vector = vectorizer.transform(['zipties zone'])
print(vector.todense())

# Selecting and Fitting the Model

We have converted our training data to a feature matrix, we can now select our model and fit it to the data

In [ ]:
from sklearn.linear_model import LogisticRegression

# y_training contains the codes associated with our training narratives
y_training = df_training['INJ_BODY_PART_CD']

# we create an instance of the LogisticRegression model and set regularization to 1.0
clf = LogisticRegression(C=10)

# we fit the model to our training data (ie. we calculate the weights)
clf.fit(x_training, y_training)

Like our other outputs, we can inspect the Logistic Regression model. The most important attribute of the model is the coefficients, which show how strongly our various features are related to our various codes. Like our feature values, these coefficients are also stored in a matrix. 

In [ ]:
print(clf.coef_.shape)
print(clf.coef_[0])

If we want, we can look at which codes each of these rows correspond to, and the features that have the highest weights (make that class much more likely)

In [ ]:
# print the codes recognized by the classifier
print(clf.classes_)

We can examine these weights more closely and see which inputs they correspond to

In [ ]:
# select the index of the code whose weights we want to examine 
# index 0 corresponds to code 100 (head not elsewhere classified)
# index 3 corresponds to code 121 (ear, external)
code_index = 0
code = clf.classes_[code_index]

# Retrieve the weights for the specified code_index
code_weights = clf.coef_[code_index]

# find the feature_index with largest weight for this code
feature_index = code_weights.argmax()

# create a dictionary that maps from index, to word
feature_mapper = {v: k for k, v in vectorizer.vocabulary_.items()}

# map that index to the word it represents
word = feature_mapper[feature_index]
print('The word with the heighest weight for code %s is "%s"' % (code, word))
print('It has a weight of:', clf.coef_[code_index, feature_index])

# Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Convert the validation narratives to a feature matrix
x_validation = vectorizer.transform(df_validation['NARRATIVE'])

# Generate predicted codes for our validation narratives
y_validation_pred = clf.predict(x_validation)

# Calculate how accurately these match the true codes
y_validation = df_validation['INJ_BODY_PART_CD']
accuracy = accuracy_score(y_validation, y_validation_pred)
macro_f1 = f1_score(y_validation, y_validation_pred, average='macro')
print('accuracy = %s' % (accuracy))
print('macro f1 score = %s' % (macro_f1))

What if we had been foolish and evaluated our performance on the training data?

In [ ]:
y_training_pred = clf.predict(x_training)
accuracy = accuracy_score(y_training, y_training_pred)
macro_f1 = f1_score(y_training, y_training_pred, average='macro')
print('accuracy = %s' % (accuracy))
print('macro f1 score = %s' % (macro_f1))

# Experimenting with Different Regularization

In [ ]:
clf = LogisticRegression(C=1)
clf.fit(x_training, y_training)

y_training_pred = clf.predict(x_training)
training_accuracy = accuracy_score(y_training, y_training_pred)
print('accuracy on training data is: %s' % training_accuracy)

y_validation_pred = clf.predict(x_validation)
validation_accuracy = accuracy_score(y_validation, y_validation_pred)
print('accuracy on validation data is: %s' % validation_accuracy)

# Experimenting with Inputs

### minimum document frequency
By default, CountVectorizer includes any feature (i.e. word) that occurs in our training data as a feature in our feature vector. We can use the min_df argument to specify that features should only be included in the feature vector if they occur in at least X different documents. Below, we specify that a feature should only be included if it occurs in at least 2 different documents. Previously this resulted in a feature vector that included 12,336 features. Now, we only get 7,163.

In [ ]:
vectorizer1 = CountVectorizer(min_df=2)
x_training = vectorizer1.fit_transform(df_training['NARRATIVE'])
x_training.shape

### n-grams
Using just the occurrence of individual words (unigrams) loses word order, which might be useful. We can counteract that by using multiple word sequences, such as bigrams (2 word sequences), trigrams (3 word sequences), and so on. In practice it is rarely useful to go beyond bigrams for text classification.

In [ ]:
vectorizer2 = CountVectorizer(min_df=5, ngram_range=(1,2))
vectorizer2.fit(df_training['NARRATIVE'])
print(len(vectorizer2.vocabulary_))

On a typical machine learning project you will likely spend a lot of time tweaking your model and retraining it, as we have done above. Eventually you will settle on the best version (as measured against your validation data). You will then measure it's performance on the test data to get a final measure of performance.

In [ ]:
x_training = vectorizer2.transform(df_training['NARRATIVE'])
clf = LogisticRegression(C=1)
clf.fit(x_training, y_training)
y_training_pred = clf.predict(x_training)
training_accuracy = accuracy_score(y_training, y_training_pred)
print('accuracy on training data is: %s' % training_accuracy)

x_validation = vectorizer2.transform(df_validation['NARRATIVE'])
y_validation_pred = clf.predict(x_validation)
validation_accuracy = accuracy_score(y_validation, y_validation_pred)
print('accuracy on validation data is: %s' % validation_accuracy)

# Saving the Model

Suppose we have settled on our final autocoder. This may have taken quite a bit of time to train and develop. Some models can take days, weeks, or even months to train. To save it for later, we need to save both the vectorizer used to encode our data and the model used to classify it. Below, we do this using the Python module `joblib`.

In [ ]:
from sklearn.externals import joblib

# save the classifier object as LRclf.pkl
joblib.dump(clf, filename='LRclf.pkl')
# save the vectorizer object as vectorizer.pkl
joblib.dump(vectorizer2, filename='vectorizer.pkl')

Later, we can load these back as shown below. Note: we must first import any objects or functions used by our pickled files, otherwise they will fail to load.

In [ ]:
from sklearn.externals import joblib
# import the objects used by our saved vectorizer and classifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

# load the classifier
clf = joblib.load(filename='LRclf.pkl')
# load the vectorizer
vectorizer = joblib.load(filename='vectorizer.pkl')

# Using the Autocoder

There are a number of ways we might use our newly created autocoder. The simplest option is to just automatically assign the codes. Suppose for example, that our test dataset is really just a set of uncoded narratives. We might code it as follows:

In [ ]:
x_test = vectorizer.transform(df_test['NARRATIVE'])
y_test_pred = clf.predict(x_test)
print(x_test.shape)
print(y_test_pred.shape)
print(y_test_pred)

### assigning codes

In [ ]:
df_test.loc[:, ('Autocode')] = y_test_pred
df_test.head(10)

### accessing the predicted probabilities

In [ ]:
y_pred_prob = clf.predict_proba(x_test)
print('The shape of the pred_prob matrix is: %s' % str(y_pred_prob.shape))
print('The probabilities for the first example are:\n%s' % y_pred_prob[0])
max_index = y_pred_prob[0].argmax()
code = clf.classes_[max_index]
print('The highest probability is at index %s which corresponds to code %s' % (max_index, code))

### adding the predicted probability for each code

In [ ]:
# get a sequence indicating the position with the highest probability for each row
top_positions = y_pred_prob.argmax(axis=1)
top_probabilities = y_pred_prob[np.arange(len(top_positions)), top_positions]
print(top_probabilities.shape)
df_test.loc[:, ('Probability')] = top_probabilities
df_test.head()

### saving the results

In [ ]:
df_test.to_excel('msha_autocoded.xlsx')

# Extras

Scikit-learn provides a wide variety of powerful tools that we didn't have time to cover in this class, including many different learning algorithms and utilities for feature and model selection. To learn more, see the official and very extensive [online documentation](http://scikit-learn.org/stable/index.html).